In [ ]:
import pprint
from pathlib import Path

import numpy as np
import tifffile
from careamics_portfolio import PortfolioManager
from itkwidgets import compare, view
from matplotlib.pyplot import imshow

from careamics_restoration.engine import Engine

### Import Dataset Portfolio


In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

Read the specific dataset 
bla 

In [ ]:
# Download files
root_path = Path("data")
files = portfolio.denoising.Flywing.download(root_path)
print(f"List of downloaded files: {files}")

In [ ]:
data_path = Path(root_path / "denoising-Flywing.unzip")

data_path.mkdir(parents=True, exist_ok=True)

### Visualize the data

In [ ]:
train_image = tifffile.imread(next(iter(data_path.rglob("*.tif"))))
print(f"Train image shape: {train_image.shape}")
imshow(np.max(train_image, axis=0), cmap="magma")

### [Optional] Visualize the data in 3D

In [ ]:
# Import itkwidets, pip install itkwidgets, if necessary

from itkwidgets import compare, view

In [ ]:
# View 3D image

view(train_image)

### Initialize the Engine

Engine contains the dataloading pipeline and the model training logic. We'll initialize the engine with the config file, but it can also be initialized from a pre-trained checkpoint.

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = Engine(config_path="n2v_3D.yml")

### Visualize training configuration

In [ ]:
pprint.PrettyPrinter(indent=2).pprint(engine.cfg.model_dump(exclude_optionals=False))


Start training

In [ ]:
train_stats, val_stats = engine.train(train_path=data_path, val_path=data_path)

### Visualize statistics

In [ ]:
import matplotlib.pyplot as plt

plt.plot([next(iter(d.values())) for d in train_stats], label="Train loss")
plt.plot([next(iter(d.values())) for d in val_stats], label="Validation loss")
plt.legend(loc="best")
plt.xlabel("Epoch")

Predict

In [ ]:
preds = engine.predict(input=data_path, tile_shape=[32, 64, 64], overlaps=[24, 48, 48])

### Visualize predictions


In [ ]:
print(f"Train image shape: {preds.shape}")
imshow(np.max(preds.squeeze(), axis=0), cmap="magma")

### [Optional] Visualize predictions in 3D

In [ ]:
compare(train_image, preds.squeeze())

### Save predictions

In [ ]:
tifffile.imwrite("preds.tif", preds.squeeze())

### Export to bioimage.io

In [ ]:
engine.save_as_bioimage(engine.cfg.experiment_name + "bioimage.zip")